# The Zebra puzzle

The Einstein Zebra puzzle, wich [according to Wikipedia](https://en.wikipedia.org/wiki/Zebra_Puzzle) is actually a puzzle from _Life International_, is stated as follows:

1. There are five houses.
2. The Englishman lives in the red house.
3. The Spaniard owns the dog.
4. Coffee is drunk in the green house.
5. The Ukrainian drinks tea.
6. The green house is immediately to the right of the ivory house.
7. The Old Gold smoker owns snails.
8. Kools are smoked in the yellow house.
9. Milk is drunk in the middle house.
10. The Norwegian lives in the first house.
11. The man who smokes Chesterfields lives in the house next to the man with the fox.
12. Kools are smoked in the house next to the house where the horse is kept.
13. The Lucky Strike smoker drinks orange juice.
14. The Japanese smokes Parliaments.
15. The Norwegian lives next to the blue house.
16. Now, who drinks water? Who owns the zebra?

In the interest of clarity, it must be added that each of the five houses is painted a different color, and their inhabitants are of different national extractions, own different pets, drink different beverages and smoke different brands of American cigarets [sic]. One other thing: in statement 6, right means your right.

— Life International, December 17, 1962

# Constaint Satisfaction Problems

The Zebra Puzzle is one of a type of problems that are more generally known as a [_contraint satisfaction problems_](https://en.wikipedia.org/wiki/Constraint_satisfaction_problem), which also includes popular puzzles like Sudoku and KenKen puzzles among many others. There are different ways of solving them, but for this puzzle we will develop a simple, straight forward algorithm.

First we need to answer: "Without any constaints, how many possible solutions are there?"
There are 5 houses, with a color, a nationality, a pet, a favorite drink an a favorite smoke. So each of these has $5!=120$ possibilities. The total number of ways of arranging the solutions is then $(5!)^5$

Q1: Compute this number.

In [5]:
# You only need the math module to do this.
import math
# Your answer here.

## Setting up a simple method to solve the problem.

### Enumeration
For a computer, numbers are easier to deal with than strings. For a human, strings are more readable, so a common way to accomodate both is called _enumeration_. This sets an aptly named variable to a number, so that in your program you can use the variable name instead of that number. You can do so by hand with:

In [2]:
Englishman = 0
Spaniard = 1

This works, however it is tedious and prone to errors. An alternative is to convert a list of strings to enumerated variables automatically. To do this, the Python `exec()` function is very useful. This function will execute the argument as if it was a program statement. Thus `exec("print('Hello'")` will just print Hello to the screen. That is not so useful, but `exec(my_command)` _is_ because now the string `my_command` can contain anything, including a complete program. 

To do the enumeration of the nationalities, we can thus do:

In [6]:
# Enumerate the nationalities:
Nationalities = "Englishman Spaniard Ukrainian Norwegian Japanese".split()  
# The string.spit() will split the string into a list of individual string at the space separator.
# Test:
print(Nationalities)
# Now turn this into 5 numeric assignments. The enumerate() function is useful:
for number,name in enumerate(Nationalities):
    exec("{}={}".format(name,number))  # The string.format() allows you to format arguments (i.e variables) into the string.

print("Number for Norwegian = ",Norwegian)    


['Englishman', 'Spaniard', 'Ukrainian', 'Norwegian', 'Japanese']
Number for Norwegian =  3


For our puzzle, this is sufficient. In a larger, more complicated program, you should probably consider making the enumerations part of a _class_, so that you use it as `Nation.Norwegian`, instead of simply `Norwegian`. It is more typing, but it clarifies what is what.

Write a minimum number of lines of code to assign numbers to the house number, colors, the pets, the drinks and the smokes. 

In [7]:
# Enumerate all items in the puzzle.
#
# Try using only one double for loop.

### Permutations
To solve the problem, we simply want to try all possible solutions against our set of rules. To do this easily, we need something that will give us all the permutation of each set of items. This can be programmed by hand, but there is a handy function that will do this for you: `itertools.permutations()`. This function will return all possible permutations of the list you give it, but it does so as an _iterator_. An iterator is a special function in Python that gives a new result each time it is called, until all results are exhausted. The `range()` function is a Python iterator (but only in Python3, it Python2 it returned a list.)

Here is an example:

In [12]:
from itertools import permutations
for i in permutations(["A","B","C"]):
    print(i)

('A', 'B', 'C')
('A', 'C', 'B')
('B', 'A', 'C')
('B', 'C', 'A')
('C', 'A', 'B')
('C', 'B', 'A')


So then cycling through all possible permutations in our problem will look something like:

In [ ]:
for color in permutations(range(5)):
    for nation in permutations(range(5)):
        for drink in permutations(range(5)):
            for smoke in permutations(range(5)):
                for pet in permutations(range(5)):
                    # Test is all conditions are met here.
                    pass # This statement does nothing, except make the loops valid.

Executing this 5 loop code will take a really long time. More than 10 seconds at least (1472 seconds, actually), which is about the amount of patience I have. Adding the test conditions where it is indicated above would only make it slower!

The trick to making this a reasonable code is the _add the test conditions as early as possible_. So, condition 10, "The Norwegian lives in the first house.", only requires the "nation" to be populated, and may go first, or "The green house is immediately to the right of the ivory house." only requires colors.

The modified loops then look like:

In [ ]:
for color in permutations(range(5)):
    # Tests requiring only color.
    for nation in permutations(range(5)):
        # Tests that involve only nation, or nation and color.
        for drink in permutations(range(5)):
            # Tests that require drink, nation, color.
            for smoke in permutations(range(5)):
                # Tests that require smoke, drink, nation, color. 
                for pet in permutations(range(5)):
                    # Test is all conditions require pet in combination with any other.
                    pass # This statement does nothing, except make the loops valid.

Depending on the details of the problem, you could get a speed gain by ordering the loops differently, i.e. perhaps do the nation loop before the color loop, but this gets into unnecessary detail here.

### Tests

How do we write the actual tests?

Well, each of the color, nation, etc variables are actually lists containing the numbers 0 through 4 ( 5 numbers) in some permutation. The code cycles through all permutations for you. If that number refers to the item (nationality, color, preferred drink, etc) of interest, and the location of the number refers to the number of the house, then we can setup test for what is in which house. 

"Milk is drunk in the middle house" then translates to:  `drink.index(Milk) == Three`, since Three is the middle house (but since we start counting at 0, the number is actually two). Note the use of the `list.index()` function, with returns the location in the list where the argument first occurs. So `drink.index(Milk)` returns the house number of the milk drinker _for this particular permutation_. It does not make sense to continue other permutations if this condition is not met.

The first condition "The Englishman lives in the red house." then translates to: `nation.index(Englishman) == color.index(Red)`. The loops then starts to look like:

In [ ]:
for color in permutations(range(5)):
    # Tests requiring only color.
    for nation in permutations(range(5)):
        # Tests that involve only nation, or nation and color.
        if nation.index(Englishman) == color.index(Red):  # Only continue if rule #2 is true.
            for drink in permutations(range(5)):
                # Tests that require drink, nation, color.
                if drink.index(Milk) == Three:            # Only continue if rule #9 is true.
                    for smoke in permutations(range(5)):
                        # Tests that require smoke, drink, nation, color. 
                        for pet in permutations(range(5)):
                            # Test is all conditions require pet in combination with any other.
                            pass # This statement does nothing, except make the loops valid.

Just these two conditions already bring down the overall execution time to something more manageable (60 seconds for  my system.)

Add all the other conditions to the set of loops.

In [13]:
#
# Complete loops and tests
#

### Printing the results

You now want to print the results in a human readable form. For the computer, the answer is encoded as numbers, which makes it difficult to figure out if you were to just print the "nation" etc variables. Fortunately, we saved the translation key in the lists at the start of this project. To translate the Englishman value back to the string "Englishman", we can again do a lookup:

In [14]:
print(Nationalities[Englishman])

Englishman


You would want to use constant spacing, make a nice table with one column for each house. The spacing can be added correctly using the `string.format()`, indicating the space in the "{}" fields of the string. So to print one of the permutations of nationalities, with a space of 20 characters each, you would do:

In [16]:
for nation in permutations(range(5)):
    # All other loops and tests are skipped.
    for n in nation:
        print("{:20s}".format(Nationalities[n]),end="") # The end="" says, don't end with a line return.
    print("")
    break       # Jump out of the loop. I want only one permutation, not all 120

Englishman          Spaniard            Ukrainian           Norwegian           Japanese            


Now you can complete the program that solves the Zebra puzzle!

Did you get the same answer as in the Wikipedia article?